<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/book_processing_gpt_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install chardet
%pip install ftfy
%pip install torch
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [80]:
from openai import OpenAI
import os
import re
import uuid
import json
import pandas as pd
from IPython.display import display
from ipywidgets import HTML
from ftfy import fix_encoding

client = OpenAI()

In [2]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    return encoding


In [3]:
filenames = []

for filename in os.listdir('books'):
    filenames.append(filename)

print(f'Books in list: {len(filenames)}')

Books in list: 43


In [245]:
def clean_answer_gpt(answer):
    system_message = {
        "framing": "You are a document processing expert. You will receive a block of text that may be an answer to an unspecified question, or may contain an answer to an unspecified question as a substring within the larger string.",
        "instructions" : "You will determine whether the string or any substring of that string contains an answer that meets the provided criteria. Consecutive sentences that address similar concepts should not be split apart. Under no circumstances should you omit a complete sentence which is related to the topic of the answer. Most 'answers' you recieve will either already be formatted as an appropriate answer, or will contains no appropriate answer. If the content is classified as a 'good question' you will return the appropraite string or substring and ONLY the string or substring. Do not provide any commentary or insight. If the provided text is not or does not contain an answer that meets these criteria you will return only the text 'bad answer' and ONLY this text. Do not provide any commentary or insight. Err on the side of rejecting answers. A false positive (incorrectly rejected good answer) is better than a false negative (incorrectly accepted bad answer).",
        "attributes": "In order to be a good answer, the answer must:\n- Be logically coherent\n- Start at the beginning of a thought and contain the full thought\n- Be grammatically correct\n- Be an answer that a guru or advisor may give during the course of a regular conversation"
    }
    user_message = {
        "context" : f"Please determine if this text is in itself a cohesive answer to a theoretical question, or if it contains a substring that is a cohesive answer to a theoretical question: {answer}",
        "attributes" : "Use this as a heuristic: Only answers that a guru or life advisor might give to a student or follower should be considered as 'good' answers. That means that the answer is coherent, complete, grammatically correct, and insightful. ",
        "instructions" : "If you identify a 'good' answer return only the string or substring of the answer. Consecutive sentences that address similar concepts should not be split apart. Only extract a substring in very clear cases. The answer you return should include all complete, gramatically correct, and topically related sentences included in the original text. You should return the longest possible 'good' answer. It is absolutely important that you ONLY return the identified answer string or substring, do not include your own commentary or any other text. The ONLY exception to this is that if there is no 'good' answer return only the string 'bad answer' and ONLY this text. No commentary."

    }
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role" : "system",
                "content" : f"{system_message["framing"]} {system_message["instructions"]} {system_message["attributes"]}" 
            },
            {
                "role" : "user",
                "content" : f"{user_message["context"]} {user_message["attributes"]} {user_message["instructions"]}"
            }

        ]
    ) 
    return response   

In [260]:
#generate quetsions to the "answers" extracted from the text
def question_generator(answer):
    system_message = {
    "framing": "You are a data enrichment specialist. You will receive a string which is an answer to an unspecified question. You must then create a question which would generate this answer in accordance with the specific attributes outlined below. The answer you have received was given by a guru or life coach in conversation with their pupil. THE MOST IMPORTANT ASPECT OF YOUR TASK IS THAT YOU GENERATE A QUESTION TTHAT SOMEONE WOULD REASONABLY ASK ALAN WATTS, AND IN A WAY THAT THE ANSWER YOU RECIEVE IS ONE THAT ALAN WATTS WOULD GIVE TO THIS QUESTION. The question you will return was one which would have been asked by their pupil. You will return only the new string you have created.",
    "attributes": "The questions should:\n- Be about THEMSELVES. X is happening to me how can I do Y, or Why do I feel like X\n- Not be something with a common sense answer\n- Relate to the answer figuratively, not literally\n- Get at a root challenge or issue without speculating about solutions\n- Use common unpretentious phrasing and vocabulary\n- Be open-ended\n- NEVER imply knowledge of the answer by using vocabulary or themes in the answer\n- Address personal growth, practical advice, philosophy, epistimology, or culture\n- Be written in the style of someone asking a question to ChatGPT (irreverant, lazy, lacking complex vocabulary)\n- Focus on experiences, challenges, concepts, and social or cultural ideas\n- Not repeat phrases from the provided answer\n- Ensure the question is logically sound.",
    "negative_response": "If the provided answer is:\n- Not suitable for any question that meets these criteria\n- Not phrased in a grammatically correct way\nNot a feasible answer to a question someone might ask a guru or life advisor\nYou MUST respond with 'bad answer'. The threshold for what constitutes a good answer should be high and if the text is not an answer to a question, you MUST respond with 'bad answer'"
    }

    user_message = {
    "framing": f"Given the following answer, generate a question that matches it:\n\nAnswer: {answer}\n\n",
    "attributes": "The question should:\n- Have an interrogative word that is appropraite for question being asked \n- Start with the situation in the asker's life that prompted the question if an applicable example might exist\n- About the asker themselves in all cases except for where this would prevent the question from making sense. 'I' instead of 'one' or 'people'\n- Simplified to the most core aspect of the question being asked. Include only enough text to make up a viable question. Additional concepts should never be chained onto the sentence with 'and'\n- Be phrased like someone asking ChatGPT a question. They should be unceremonious, brief, and not use contrived language\n- Be open-ended\n- Where appropriate, seek guidance or insight on a question that most people grapple with\n- Naturally curious\n- Not leading. Any usage of vocabulary or themes from the answer will ruin the realism of the answer-question pair\n- Deal with worldly issues of regular people. For example: personal betterment, practical advice, or questions about cultural and social issues\n\n Wherever possible the questions should be written in layperson language in such a way that the answer is an appropriate metaphorical rather than literal answer. It is unlikely that a pupil will ask a pionted enough question to where the answer would be a direct literal answer to their question.",
    "negative_response": "If the provided answer is:\n- Not suitable for any question that meets these criteria\n- Not phrased in a grammatically correct way\nNot a feasible answer to a question someone might ask a guru or life advisor, you MUST respond with 'bad answer'."
    }
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system",
                #"content": "You are assisting in the generation of training data for fine-tuning. You will receive a chunk of text, and will respond with a short casually phrased question to which the chunk of text you received would be an expected answer. The person who generated the answer is Alan Watt's and often he will give a response that answers a question only indirectly. The question should be opened ended, NO leading questions. If the text you receive is not an answer to a question which would be asked in normal speech respond with 'bad answer'."
                #gpt suggestion
                #"content": "You are an assistant that generates questions to match provided answers. The questions should:\n- Be open-ended\n- Naturally curious\n- Avoid implying knowledge of the answer\n- Simple and concise. Meaning single-part, no 'and' additions\n- Address personal growth, practical advice, or self-awareness\n- Focus on experiences, challenges, or motivations\n- Explore options, perspectives, or solutions\n\nIf the provided answer is not suitable for any question that meets these criteria or is not phrased in a grammatically correct way, respond with 'bad answer'."
                "content" : system_message["framing"] + system_message["attributes"] + system_message["negative_response"]
            },
            {
                "role": "user",
                #"content": f"Return a short question to which this text would be an appropriate answer. Do not directly repeat the key words in the answer. This is a question that someone would be asking their spiritual guru. They should be situational in nature, dealing with the real life issues the asker is facing. ABSOLUTELY NO RHETORICAL, LEADING, OR SELF-ANSWERING QUESTIONS. If the text you receive is not an answer to a question which would be asked in normal speech respond with 'bad answer'. : {answer}"
                #gpt suggestion
                "content": user_message["framing"] + user_message["attributes"] + user_message["negative_response"]
            }
        ]
    )
    return response

In [74]:
#split chunks into sections that could reasonably be the answer to a question
def chunk_gpt(text):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {
                "role": "system",
                #chatGPT suggestion 1 - "content": "You are a document processor used to create training data for fine-tuning. You will receive a paragraph from a book. The singular goal of your outputs is to identify strings of text that could be the answer to an unspecified question. To do this, you will determine if the paragraph itself is a cohesive answer, if it contains multiple cohesive answers, or if it contains no answers at all. You will return the list of so-called answers separated by line-breaks in the case that they exist, and the string 'bad question' in the case that they do not. Direct, concrete, or factual answers are bad answers, do not include them."
                #chatGPT suggestion 2
                "content" : "You are a document processor used to create training data for fine-tuning. You will receive a paragraph from a book. Your goal is to identify strings of text that could serve as answers to theoretical, insightful questions. These answers should be cohesive, maintaining the depth and nuance of the original writing. Avoid short, fragmented phrases. Return the list of cohesive answers separated only by line breaks, and if no such answers exist, return 'bad answer'. Direct, concrete, or purely factual answers are bad answers; do not include them."
                #"content": "You are a document processor used to create training data for fine-tuning. You will receive a paragraph of a book. The singular goal of your outputs is to identify strings of text that could be the answer to an unspecified question. To do this you will determine if the paragraph itself is a cohesive answer, if it contains multiple cohesive answers, or if it contains no answers at all. You will return the list of so-called answers separated by line-breaks in the case that they exist and the string 'bad question' in the case that they do not. Direct concrete or factual answers are bad answers, do not include them."
                #"content": "You are a document processor used to create fine-tuning data. You will receive a paragraph from a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. In many or even most cases the text will not need to be split at all. The cohesive answers within the text may be directly following each other and there may be space between them that needs to be removed. If the text cannot be reasonably split, and could not be interpreted as the answer to a theoretical question respond with only 'bad question'. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks with the exception of bad answers. Minor grammatical may be made as needed."
            },
            {
                "role": "user",
                #"content": f"Return the line-break separated list of cohesive 'answers' in this text. Do not split the text if it is not necessary :\n\n{text}"
                #chatGPT suggestion
                "content": f"Extract content from the following paragraph that could serve as a reasonable answer to a theoretical question. Provide multiple strings if possible separated only by line breaks. Ensure proper punctuation and capitalization at the beginning and end of sentences. Do not alter phrasing or word order IN ANY WAY. \n\n Paragraph: {text}"
            }
        ]
    )
    return response.choices[0].message.content.strip().split('\n')

In [34]:
#loop through all of the processed "answers" and generate questions
def process_questions(processed_answers, book_name):
    for i in range(len(processed_answers)):

        #limiting cycles for testing
        if i > 2 and debugger == True:
            continue

        answer = processed_answers[i]
        #remove blanks
        if len(answer) < 30:
            continue

        question = answer_gpt(answer)

        json_obj = {
            "custom-id" : f"{book_name}-{str(uuid.uuid4())}",
            "instruction" : "You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish. If a question is silly, factual/objective, or is not something that someone would reasonably ask Alan Watts do not answer it.",
            "input" : question,
            "output" : answer
        }

        processed_json.append(json_obj)

In [63]:
#loop through the chunks of a book
def process_book_chunks(text_chunks, book_name):
    print(f'Processing {len(text_chunks)} chunks for {book_name}')
    for i in range(len(text_chunks)):
        chunk = fix_encoding(text_chunks[i])

        #skip the first and last pages which are usually credits and other misellaneous content
        if i > len(text_chunks)-8 or i < 8:
            continue

        #limiting requests for testing purposes
        if i > 9 and debugger == True:
            continue

        processed_answers = chunk_gpt(chunk)
        process_questions(processed_answers, book_name)
    print(f'Completed processing {book_name}')

In [ ]:
raw_chunk_list = []
process_books()

In [77]:
#process books into chunks of characters

def process_books(debugger):
    book_status = display('Initializing..', display_id=True) 
    #chunk_size = 5000
    for i in range(len(filenames)):

        #limiting cycles for testing
        if i > 10 and debugger == True:
            continue 

        encoding = detect_encoding(f'books/{filenames[i]}')
        with open(f'books/{filenames[i]}', 'r', encoding=encoding, errors='replace') as file:
            book_status.update(HTML(f"Processing {filenames[i]}... <br>{len(raw_chunk_list)} total text blocks"))
            
            pattern_lineb = re.compile(r'\n(?!\n)') #match linebreaks
            pattern_colonb = re.compile(r'(?<=:)(\\n)') #match colons followed by examples
            pattern_pageb = re.compile(r'\n(?=[a-z])') #match pagebreaks, where a linebreak is followed by a lowercase character
            pattern_chapter_titles = re.compile(r'(?<=\n)([A-Z]| |\d){1,15}(?=\n)') #matches chapter titles
            pattern_book_name_clean = re.compile(r'_|\(.*\)_djvu.txt') 
            pattern_book_name = re.compile(r'(?<=- ).*(?= \()')

            raw_book_contents = file.read()

            #while pagebreaks exist, remove them
            while (re.search(pattern_pageb, raw_book_contents) or re.search(pattern_colonb, raw_book_contents)):
                raw_book_contents = pattern_chapter_titles.sub('', raw_book_contents)
                raw_book_contents = pattern_pageb.sub('', raw_book_contents)
                raw_book_contents = pattern_colonb.sub('', raw_book_contents)
                
            bookname = filenames[i] #re.match(pattern_book_name, filenames[i]) if re.match(pattern_book_name, filenames[i]) else pattern_book_name_clean.sub(' ', filenames[i])
            if (re.search(pattern_book_name, bookname)):
                bookname = re.search(pattern_book_name, bookname)[0]

            bookname =  pattern_book_name_clean.sub('', bookname.replace('-',' ').strip())

            contents = pattern_lineb.sub('', raw_book_contents).split('\n')
            
            text_chunks = [
                [fix_encoding(paragraph.replace('\n','')), bookname]  
                for paragraph 
                in contents 
                if paragraph 
                and paragraph.strip() 
                and len(paragraph) > 100 
                and not re.compile(r'\d{2,}|publi|watts|alan', re.IGNORECASE).search(paragraph)
            ]
            #print(text_chunks)
            raw_chunk_list.extend(text_chunks)
            # for chunk in text_chunks:
            # there are raw chunks here, which have been filtered to exclude bad data    
    
    book_status.update(HTML(f"Processing complete <br>{len(raw_chunk_list)} total text blocks"))
    print(json.dumps(raw_chunk_list, indent=4))

In [ ]:
raw_chunk_list = []
process_books(debugger=False)

In [81]:
df = pd.DataFrame(raw_chunk_list)
df.to_json('paragraphs_20240805_01.json', orient='records', index=False, lines=True, force_ascii=False)

In [302]:
def generate_complete_pairs(json_list, debugger, range_start, range_end):
    total_completion_tokens = 0
    total_prompt_tokens = 0
    pairs_processed = 0
    bad_answers_caught = 0
    handle = display("Initializing...", display_id=True)

    for i in range(len(raw_chunk_list)):
        if debugger and (i < range_start or i > range_end):
            continue
        text_response_raw = clean_answer_gpt(raw_chunk_list[i][0])
        text_response_clean = text_response_raw.choices[0].message.content

        if text_response_clean == 'bad answer':
            bad_answers_caught += 1
            continue
        
        question_response_raw = question_generator(text_response_clean)
        question_response_clean = question_response_raw.choices[0].message.content

        pairs_processed += 1
        total_completion_tokens += (text_response_raw.usage.completion_tokens + question_response_raw.usage.completion_tokens)
        total_prompt_tokens += (text_response_raw.usage.prompt_tokens + question_response_raw.usage.prompt_tokens)

        json_obj = { 
                "book" : f"{raw_chunk_list[i][1]}-{str(uuid.uuid4())}",
                "instruction" : "You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.", 
                "input" : question_response_clean, 
                "output" : text_response_clean 
        }

        json_list.append(json_obj)

        handle.update(HTML(f"pairs processed: {pairs_processed}<br>bad answers caught: {bad_answers_caught}<br> completion tokens: {total_completion_tokens}<br>prompt tokens: {total_prompt_tokens}<br>cost: ${((total_completion_tokens/1000000)*1.5)+((total_prompt_tokens/1000000)*.5)}"))

In [ ]:
generate_complete_pairs(json_list, True, 1001, 8000)

In [313]:
import pandas as pd
df = pd.DataFrame(json_list)
df.to_json('1000_8000_pairs.json', orient='records', index=False, lines=True, force_ascii=False)

In [ ]:
print(json.dumps(json_list, indent=4, ensure_ascii=False))      

In [87]:
def chunk_entry(chunk, book):
    bulk_entry = { "custom_id": f"{book}-{str(uuid.uuid4())}", "method": "POST", "url": "/v1/chat/completions", "body": { "model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a document processor used to create fine-tuning data. You will receive a paragraph from a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may be directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed."},{"role": "user", "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question and separate them with line breaks. :\n\n{chunk}"}],"max_tokens": 5000}}
    chunk_list.append(bulk_entry)